Get images and save them independently as 2D slices

Dependencies

In [1]:
'''
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons

import scipy
'''
import SimpleITK as sitk
reader = sitk.ImageFileReader()
reader.SetImageIO("MetaImageIO")

import numpy as np

import os

import pathlib

from natsort import natsorted

'''
#Set GPU/Cuda Device to run model on
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
'''
np.random.seed(46)

Dataset Directories

In [2]:
#Toy Dataset Directory from Spider | Grand Challenge 
dummy_train_img_dir = pathlib.Path(r"spider_toy_dset/dummy_train_images")
dummy_train_label_dir = pathlib.Path(r"spider_toy_dset/dummy_train_labels")
dummy_test_img_dir = pathlib.Path(r"spider_toy_dset/dummy_test_images")
dummy_test_label_dir= pathlib.Path(r"spider_toy_dset/dummy_test_labels")

#Full Dataset Spider Local Directory 


Image Class, Returns 3D Array

In [7]:
from transforms import mri_transforms


#TODO add bool image label for interp 
class Mri:
    def __init__(self, path, is_label, is_train_set):
        mri_mha = sitk.ReadImage(path, imageIO = "MetaImageIO") #explicitly setting ioreader just in case
        
        #resampling voxel spacing
        if(is_train_set):
          mri_mha = mri_transforms.resample_img(mri_mha, out_spacing= [2.0, 0.6, 0.6], is_label=is_label) #BSpline interp
      
         
        #get 3d array from mri
        mri_a = np.array(sitk.GetArrayFromImage(mri_mha)) #mri_array

        #transpose array to format z x y 
        #note: for some reason sitk.GetArrayFromImage() takes an image with dims [slice index, row, col] and transforms it to[row, col, slice index]
        #this bit of code solves that issue, not sure why it happens but this is enough to make sure all the images are of format [slice, row, col]
        if(mri_a.shape[0] > mri_a.shape[1] or mri_a.shape[0] > mri_a.shape[2]): #if z axis isn't first
          mri_a = np.transpose(mri_a, (2, 0, 1))
        
        
        '''
        if(mri_a.shape[2] > mri_a.shape[1]): #bring images to vertical orientation
          mri_a = np.transpose(mri_a, (0, 2, 1))
        '''
      
        mri_a_float32 = mri_a.astype(dtype = np.float32)
        #TODO: set bounds to [-1000, 2000] https://en.wikipedia.org/wiki/Hounsfield_scale
        self.hu_a = mri_a_float32

Sort Directories

In [9]:
#get lists from directories

#toy dset 

label_train_dir_list = os.listdir(dummy_train_label_dir)
image_train_dir_list = os.listdir(dummy_train_img_dir) 

#local dset
'''
image_dir_list = os.listdir(local_img_idr)
label_dir_list = os.listdir(local_label_dir)
'''
#sort lists
image_train_dir_list = natsorted(image_train_dir_list)
label_train_dir_list = natsorted(label_train_dir_list)
#empty lists to hold x and y dimensions of images
row_list = []
col_list = []

#dirlen = len(os.listdir(dummy_train_label_dir))
dirlen = len(os.listdir(dummy_train_label_dir))

print(dirlen)

#print(local_label_idr)


44


Slice Directories

In [10]:
from transforms import array_transforms

dummy_train_img_slice_dir = pathlib.Path(r"spider_toy_dset_slices/dummy_train_img_slices")
dummy_train_label_slice_dir = pathlib.Path(r"spider_toy_dset_slices/dummy_train_label_slices")
dummy_test_img_slice_dir = pathlib.Path(r"spider_toy_dset_slices/dummy_test_img_slices")
dummy_test_label_slice_dir= pathlib.Path(r"spider_toy_dset_slices/dummy_test_label_slices")







Remove slices with 0 label value and save the remaining slices in a new directory | TOY TRAIN SET <br>
DON'T RUN UNLESS EMPTY DIRECTORIES FOR THE IMAGES ARE CREATED


In [11]:
#train directories images to slices 
from transforms import array_transforms

for idx in range(0, dirlen):
  #print("dirlen", dirlen)
  
 #toy dset 
  
  img_path = dummy_train_img_dir.joinpath(image_train_dir_list[idx])
  label_path = dummy_train_label_dir.joinpath(label_train_dir_list[idx])#first part before joinpath is pathlib.Path, second part is the directory of the file 

  #print(image_train_dir_list[idx])
  #print(type(image_train_dir_list[idx]))

  '''
  img_path = local_img_idr.joinpath(image_dir_list[idx])
  label_path = local_label_dir.joinpath(label_dir_list[idx]) #first part before joinpath is pathlib.Path, second part is the directory of the file 
  '''
  image = Mri(img_path, is_label= False, is_train_set= True)
  label = Mri(label_path, is_label= True, is_train_set= True) #for resampling
  
  #print("hu_a dims", image.hu_a.shape)

  image_a = image.hu_a
  label_a = label.hu_a

  #print("last image shape before remove empty", image_a.shape)

  #remove empty slices
  image_a, label_a = array_transforms.remove_empty_slices(image_a, label_a)

  
  '''
  #zero crop
  image_a, label_a = array_transforms.crop_zero(image_a, label_a)
  '''
  
  array_transforms.extract_slices(image_a, image_train_dir_list[idx], dummy_train_img_slice_dir) 
  array_transforms.extract_slices(label_a, label_train_dir_list[idx], dummy_train_label_slice_dir) 

  #break #BREAK TESTING FOR 1 IMAGE

  #TODO GET IMAGES FROM THE RIGHT AXIS SEE PRINTS  

  #print("label after: ", label_a.shape)
  



(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)
(467, 482)

Remove slices with 0 label value and save the remaining slices in a new directory | TOY TEST SET <br>
DON'T RUN UNLESS EMPTY DIRECTORIES FOR IMAGES ARE CREATED

In [14]:
label_test_dir_list = os.listdir(dummy_test_label_dir)
image_test_dir_list = os.listdir(dummy_test_img_dir) 

#local dset
'''
image_dir_list = os.listdir(local_img_idr)
label_dir_list = os.listdir(local_label_dir)
'''
#sort lists
image_train_dir_list = natsorted(image_test_dir_list)
label_train_dir_list = natsorted(label_test_dir_list)

dirlen = len(os.listdir(dummy_test_label_dir))

print(dirlen)

#train directories images to slices 
from transforms import array_transforms

for idx in range(0, dirlen):
  #print("dirlen", dirlen)
  
 #toy dset 
  
  img_path = dummy_test_img_dir.joinpath(image_test_dir_list[idx])
  label_path = dummy_test_label_dir.joinpath(label_test_dir_list[idx])#first part before joinpath is pathlib.Path, second part is the directory of the file 

  #print(image_train_dir_list[idx])
  #print(type(image_train_dir_list[idx]))

  image_test = Mri(img_path, is_label= False, is_train_set=False)
  label_test = Mri(label_path, is_label= True, is_train_set= False)
  
  image_test_a = image_test.hu_a
  label_test_a = label_test.hu_a

  #no feature extraction on test set, leaving as is
  
  array_transforms.extract_slices(image_test_a, image_test_dir_list[idx], dummy_test_img_slice_dir) 
  array_transforms.extract_slices(label_test_a, label_test_dir_list[idx], dummy_test_label_slice_dir) 

  #print("label after: ", label_a.shape)
  


  






10
